In [1]:
import os
import sys
sys.path.append("..")

import tensorflow as tf

import cst_model as cst
import distortion_layers as ly
import base_models as bm
import callbacks as cb


tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

2022-06-23 16:32:58.307700: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 16:32:58.307719: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
dcs = {
    1:{
        "contrast": {"factor": [0.2, 0.2]},
        "color": {"factor": [5,0,5]},
        "blur": {"filter_shape": 2, "sigma": 0.},  # kernel size is 'filter_shape * 2 + 1'
        "brightness": {"lower": .7, "upper":1.3}
    },
    2:{
        "contrast": {"factor": [0.2, 2.2]},
        "color": {"factor": [5,0,5]},
        "blur": {"filter_shape": 2, "sigma": 1.},
        "brightness": {"lower": .7, "upper":1.3}
    },
    3:{
        "contrast": {"factor": [0.4, 0.6]},
        "color": {"factor": [20,0,20]},
        "blur": {"filter_shape": 2, "sigma": 5.},
        "brightness": {"lower": .5, "upper":1.5}
    },
    4:{
        "contrast": {"factor": [0.4, 0.6]},
        "color": {"factor": [20,20,20]},
        "blur": {"filter_shape": 4, "sigma": 5.},
        "brightness": {"lower": .5, "upper":1.5}
    }
}


layers = [
    ly.DistortionLayer(layers=[
        ly.RandomColorByChannel(**dcs[i]["color"]), 
        tf.keras.layers.RandomContrast(**dcs[i]["contrast"]),
        ly.RandomBrightness(**dcs[i]["brightness"]),
        ly.RandomGaussianBlur(**dcs[i]["blur"])])
    for i in dcs
]

2022-06-23 16:32:59.867512: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-23 16:32:59.867883: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-23 16:32:59.867936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-06-23 16:32:59.867983: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-06-23 16:32:59.869624: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [3]:
train_path = "../data"
tile_size = 128
batch_size = 20
channels = 3
n_st_components = 1
epochs = 10

gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = 0.2
)
flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # 'rgb' or 'grayscale' 
    batch_size=batch_size,
    class_mode='binary', # 'sparse' for multiclass, 'binary' for binary 
)


Found 52212 images belonging to 2 classes.


In [8]:
metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]

base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=1)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = cst.CSTModel(inputs=i, outputs=x_i, alpha=1, n_st_components=n_st_components, dist_layers=layers[2])


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = tf.keras.losses.binary_crossentropy,
    metrics = metrics 
)

cst_model.fit(
    x=flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(flow.classes)
)

Epoch 1/10
  61/2611 [..............................] - ETA: 59:39 - loss: 0.4502 - sce: 0.6631 - acc: 0.7459 - sce_0: 0.6631 - acc_0: 0.7459

KeyboardInterrupt: 